## Structuring reddit's comments

In the following list there are reddit submission ids chosen by hand from Google search of 'Best and worst [product_family]' where product families are iphones, android phones, backpacks and liquid foundations.

In [1]:
sub_list = ['5nrr82','chz8lg','5kl0is',
             'jzsyko','kew1py','hvlbp2','k9btlq',
             '8uaj1y','6wjz7o','5e7ap6','kx4bkg','ksbw5v','dhg7ej','axwymo']

In [2]:
import re
import praw
import pandas as pd
from utils import getAll
import pickle
import json

Connecting to reddit's API to get the comments on each submission. 

In [3]:
with open("config.json") as json_data_file:
    config = json.load(json_data_file)

username = config['username']
userAgent = config['userAgent']
clientId = config['clientId']
clientSecret = config['clientSecret']
password = config['password']
r = praw.Reddit(user_agent=userAgent, client_id=clientId, client_secret=clientSecret)


Get all the comments from each submission filtering the ones that have less than 10 upvotes.

In [4]:
all_comments = []

In [5]:
for sub in sub_list:
    all_comments.append(getAll(r,sub))

In [6]:
len(all_comments)

14

In [7]:
lengs = []
for comment in all_comments:
    lengs.append(len(comment['comments']))

In [8]:
lengs

[10, 5, 63, 23, 2, 20, 17, 98, 69, 1, 15, 2, 7, 19]

In [9]:
global links
links = 0
def extact_links_with_text(text):
    # Anything that isn't a square closing bracket
    global links
    name_regex = "[^]]+"
    # http:// or https:// followed by anything but a closing paren
    url_regex = "http[s]?://[^)]+"

    markdown_regex = '\[({0})]\(\s*({1})\s*\)'.format(name_regex, url_regex)
    matches = re.findall(markdown_regex, text)
    amazon_links = []
    if len(matches) > 0:
        links += 1
    for match in matches:
        # Replacing the link with the text
        text = text.replace(f'[{match[0]}]({match[1]})',match[0])
        if 'amazon.com' in match[1]:
            print(match[1])
            amazon_links.append(match[1])    
    commment = {'text':text,'links':matches,'amazon_links':amazon_links}
    return commment

In [10]:
global simple_links
simple_links = 0
def extract_links(comment):
    global simple_links
    url_regex = '(http|ftp|https)://([\w_-]+(?:(?:\.[\w_-]+)+))([\w.,@?^=%&:/~+#-]*[\w@?^=%&/~+#-])?'
    urls = re.findall(url_regex, comment['text'])
    for url in urls:
        full_url = ''.join(url)
        comment['links'].append(full_url)
        if 'amazon' in url[1]:
            prod_description = url[2].split('/')[1]
            comment['text'] = comment['text'].replace(full_url,prod_description)
            comment['amazon_links'].append(full_url)
        else:
            print(''.join(url).replace('https','').replace('http',''))
            comment['text'] = comment['text'].replace(full_url,'') 
        simple_links += 1
    return comment

### Link processing pipeline

- First I extract the links in markdown and replace the combination of (url)[text]  with only the text.
- Then I extract the links that are embedded en the text.
- Whenever there's an Amazon link, I store that particular link and replace with the first part of the url.
- I store each submission dictionary in a pickle file


In [12]:
for sub in all_comments:
    processed_comments = [extact_links_with_text(comment) for comment in sub['comments']]
    processed_comments = [extract_links(comment) for comment in processed_comments]
    name = '_'.join(sub['submission'].split(' ')).lower()
    with open(f'submissions/{name}.pickle', 'wb') as handle:
        pickle.dump(processed_comments, handle, protocol=pickle.HIGHEST_PROTOCOL)

24.media.tumblr.com/27b571b671a3454d1edc9bc6f4ad3c32/tumblr_n3rxlsb9MM1t9sksvo1_400.gif
www.nbcnews.com/technolog/iphone-5s-cost-make-199-iphone-5c-173-8C11256386
https://www.amazon.com/Outdoor-Backpack-Camping-Rucksack-Shoulder/dp/B00XKUQQC8/ref=sr_1_5?ie=UTF8&qid=1530120901&sr=8-5&keywords=kaukko
https://www.amazon.com/Herschel-Supply-Co-Little-America/dp/B0077BZW26/
https://www.amazon.com/AmazonBasics-ZH1508073-Amazonbasics-Classic-Backpack/dp/B013TGEJEE
www.fjallraven.co.uk/raven-20l.Its
www.vesselworkshop.com/bike-accessories/beatnik-pnchb.This
www.tumi.com/p/shaw-deluxe-brief-pack%C2%AE-0222389NVY2
www.wantlesessentiels.com/us_en/shop-men#top-bread
www.coteetciel.com/en-US/
www.youtube.com/watch?v=ReT4YkJRewA
gsmarena.com/results.php3?nYearMin=2018&nHeightMax=150
www.bricklanebikes.co.uk/sugino-pista-crank-black
